# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 56 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.04276


extracting tarball to tmp_2302.04276...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.04280


extracting tarball to tmp_2302.04280...

...

/tmp/ipykernel_2034/4030337529.py:34: LatexWarning: 2302.04671 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2302.04757


extracting tarball to tmp_2302.04757...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2302.04757/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04507-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04507) | **Euclid preparation: XXVIII. Modelling of the weak lensing angular power  spectrum**  |
|| Euclid Collaboration, et al. -- incl., <mark>M. L. Brown</mark>, <mark>E. Franceschi</mark>, <mark>K. Jahnke</mark>, <mark>S. Ligori</mark>, <mark>P. B. Lilje</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| *20 pages, submitted to A&A*|
|**Abstract**| This work considers which higher-order effects in modelling the cosmic shear angular power spectra must be taken into account for Euclid. We identify which terms are of concern, and quantify their individual and cumulative impact on cosmological parameter inference from Euclid. We compute the values of these higher-order effects using analytic expressions, and calculate the impact on cosmological parameter estimation using the Fisher matrix formalism. We review 24 effects and find the following potentially need to be accounted for: the reduced shear approximation, magnification bias, source-lens clustering, source obscuration, local Universe effects, and the flat Universe assumption. Upon computing these explicitly, and calculating their cosmological parameter biases, using a maximum multipole of $\ell=5000$, we find that the magnification bias, source-lens clustering, source obscuration, and local Universe terms individually produce significant ($\,>0.25\sigma$) cosmological biases in one or more parameters, and accordingly must be accounted for. In total, over all effects, we find biases in $\Omega_{\rm m}$, $\Omega_{\rm b}$, $h$, and $\sigma_{8}$ of $0.73\sigma$, $0.28\sigma$, $0.25\sigma$, and $-0.79\sigma$, respectively, for flat $\Lambda$CDM. For the $w_0w_a$CDM case, we find biases in $\Omega_{\rm m}$, $\Omega_{\rm b}$, $h$, $n_{\rm s}$, $\sigma_{8}$, and $w_a$ of $1.49\sigma$, $0.35\sigma$, $-1.36\sigma$, $1.31\sigma$, $-0.84\sigma$, and $-0.35\sigma$, respectively; which are increased relative to the $\Lambda$CDM due to additional degeneracies as a function of redshift and scale. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04651-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04651) | **Mass Reconstruction of Galaxy-scale Strong Gravitational Lenses Using  Broken Power-law Model**  |
|| Wei Du, et al. -- incl., <mark>Liping Fu</mark>, <mark>Ran Li</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| *22 pages, 13 figures, 1 table. Submitted to ApJ, comments welcome!*|
|**Abstract**| With mock strong gravitational lensing images, we investigate the performance of broken power-law (BPL) model on the mass reconstruction of galaxy-scale lenses. An end-to-end test is carried out, including the creation of mock strong lensing images, the subtraction of lens light, and the reconstruction of lensed images. Based on these analyses, we can reliably evaluate how accurate the lens mass and source light distributions can be measured. We notice that, based on lensed images alone, only the Einstein radii ($R_{\rm E}$) or the mean convergence within them can be well determined, with negligible bias (typically $<1\%$) and controllable uncertainty. Away from the Einstein radii, the radial and mean convergence profiles can hardly be constrained unless well-designed priors are applied to the BPL model. We find that, with rigid priors, the BPL model can clearly outperform the singular power-law models by recovering the lens mass distributions with small biases out to several Einstein radii (e.g., no more than $5\%$ biases for the mean convergence profiles within $3~R_{\rm E}$). We find that the source light reconstructions are sensitive to both lens light contamination and lens mass models, where the BPL model with rigid priors still performs best when there is no lens light contamination. It is shown that, by correcting for the projection effect, the BPL model is capable of estimating the aperture and luminosity weighted line-of-sight velocity dispersions to an accuracy of $\sim6\%$. These results further highlight the great potential of the BPL model in strong lensing related studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04276-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04276) | **The Simons Observatory: pipeline comparison and validation for  large-scale B-modes**  |
|| K. Wolz, et al. -- incl., <mark>C. Hervias-Caimapo</mark>, <mark>A. Baleato Lizancos</mark>, <mark>M. L. Brown</mark>, <mark>B. Jost</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| *Submitted to Astronomy & Astrophysics. 20 pages, 15 figures*|
|**Abstract**| The upcoming Simons Observatory Small Aperture Telescopes aim at achieving a constraint on the primordial tensor-to-scalar ratio $r$ at the level of $\sigma(r=0)\lesssim0.003$, observing the polarized CMB in the presence of partial sky coverage, cosmic variance, inhomogeneous non-white noise, and Galactic foregrounds. We present three different analysis pipelines able to constrain $r$ given the latest available instrument performance, and compare their predictions on a set of sky simulations that allow us to explore a number of Galactic foreground models and elements of instrumental noise, relevant for the Simons Observatory. The three pipelines use different combinations of parametric and non-parametric component separation at the map and power spectrum levels, and employ $B$-mode purification to estimate the CMB $B$-mode power spectrum. They are tested and compared regarding their capability to analyze a common set of simulated realistic frequency maps, and to extract constraints on the tensor-to-scalar ratio $r$. Their performance is evaluated in terms of bias and statistical uncertainty on this parameter. In most of the scenarios the three methodologies achieve similar performance. Nevertheless, several simulations with complex foreground signals lead to a $>2\sigma$ bias on $r$ if analyzed with the default versions of these pipelines, highlighting the need for more sophisticated pipeline components that marginalize over foreground residuals. We show two such extensions, using power-spectrum-based and map-based methods, that are able to fully reduce the bias on $r$ below the statistical uncertainties in all foreground models explored, at a moderate cost in terms of $\sigma(r)$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04645-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04645) | **Spectroscopy of 9 eruptive young variables using TANSPEC**  |
|| <mark>Arpan Ghosh</mark>, et al. -- incl., <mark>Saurabh Sharma</mark>, <mark>Supriyo Ghosh</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| *13 pages, 11 figures*|
|**Abstract**| In recent times, 3.6m Devasthal Optical Telescope (DOT) has installed an optical to near infra-red spectrograph, TANSPEC, which provides spectral coverage from 0.55-2.5 microns. Using TANSPEC, we have obtained a single epoch spectrum of a set 9 FUors and EXors. We have analysed line profiles of the sources and compared with the previously published spectra of these objects. Comparing the line profile shapes with the existing theoretical predictions, we have tried to interpret the physical processes that are responsible for the current disc evolution and the present accretion dynamics. Our study has shown the importance for time evolved spectroscopic studies for better understanding the evolution of the accretion mechanisms. This in turn can help in better categorisation of the young stars displaying episodic accretion behaviour. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04757-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04757) | **An In-Depth Look at TOI-3884b: a Super-Neptune Transiting a M4 Dwarf  with Persistent Star Spot Crossings**  |
|| <mark>Jessica E. Libby-Roberts</mark>, et al. -- incl., <mark>Leslie Hebb</mark>, <mark>Andrea S.J. Lin</mark>, <mark>Suvrath Mahadevan</mark>, <mark>Henry A. Kobulnicky</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| *Submitted to AJ, Comments welcome*|
|**Abstract**| We perform an in-depth analysis of the recently validated TOI-3884 system, an M4 dwarf star with a transiting super-Neptune. Using high precision light curves obtained with the 3.5 m Apache Point Observatory and radial velocity observations with the Habitable-zone Planet Finder (HPF), we derive a planetary mass of 32.6$^{+7.3}_{-7.4}$ M$_{\oplus}$ and radius of 6.4 $\pm$ 0.2 R$_{\oplus}$. We detect a distinct star spot crossing event occurring just after ingress and spanning half the transit for every transit. We determine this spot feature to be wavelength-dependent with the amplitude and duration evolving slightly over time. Best-fit star spot models show that TOI-3884b possesses a misaligned ($\lambda$ = 75 $\pm$ 10$^\circ$) orbit which crosses a giant pole-spot. This system presents a rare opportunity for studies into the nature of both a misaligned super-Neptune and spot evolution on an active mid-M dwarf. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04294-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04294) | **Microporosity and parent body of the rubble-pile NEA (162173) Ryugu**  |
|| <mark>Wladimir Neumann</mark>, et al. |
|*Appeared on*| *2023-02-10*|
|*Comments*| **|
|**Abstract**| Observations of C-type NEAs and lab investigations of carbonaceous chondritic (CC) meteorites provide strong evidence for a high porosity of C-type asteroids. Boulder microporosity values derived from in-situ measurements at the surface of the rubble-pile NEA Ryugu of up to 55% are substantially higher than for water-rich CC samples and could indicate distinct evolution paths for the parent body of Ryugu and parent bodies of carbonaceous chondrites, despite spectral similarities. In the present study, we calculate the evolution of the temperature and porosity for early solar system's planetesimals in order to constrain the range of parameters that result in microporosities compatible with Ryugu's high-porosity material and likely burial depths for the boulders observed at the surface. By varying key properties of the parent body, such as accretion time t0 and radius R that have strong influence on temperature and porosity and by comparing the interior porosity distribution with the measured boulder microporosity, hydration, and partial dehydration of the material, we constrain a field within the (R,t0)-space appropriate for bodies that are likely to have produced such material. Our calculations indicate a parent body size of only a few km and its early accretion within <~2-3 Myr after the formation of Ca-Al-rich inclusions. A gradual final porosity profile of best-fit bodies indicates production of both low- and high-density boulders from the parent body material. By contrast, parent body properties for CI and CM chondrites obtained by fitting carbonate formation data indicate a radius of ~20-25 km and an accretion time of ~3.75 Myr after CAIs. These results imply a population of km-sized early accreting porous planetesimals as parent bodies of Ryugu (and, potentially, other NEAs) and a population of larger late accreting less porous parent bodies of water-rich carbonaceous chondrites. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04671-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04671) | **In-flight Calibration of the Magnetometer on the Mars Orbiter of  Tianwen-1**  |
|| Zhuxuan Zou, et al. -- incl., <mark>Tielong Zhang</mark>, <mark>Guoqiang Wang</mark>, <mark>Zhoubin Zhang</mark>, <mark>Yiren Li</mark>, <mark>Kai Liu</mark>, <mark>Jingnan Guo</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| **|
|**Abstract**| Mars Orbiter Magnetometer (MOMAG) is one of seven science payloads onboard Tianwen-1's orbiter. Unlike most of the satellites, Tianwen-1's orbiter is not magnetically cleaned, and the boom where placed the magnetometer's sensors is not long enough. These pose many challenges to the magnetic field data processing. In this paper, we introduce the in-flight calibration process of the Tianwen-1/MOMAG. The magnetic interference from the spacecraft, including spacecraft generated dynamic field and slowly-changing offsets are cleaned in sequence. Then the calibrated magnetic field data are compared with the data from the Mars Atmosphere and Volatile EvolutioN (MAVEN). We find that some physical structures in the solar wind are consistent between the two data sets, and the distributions of the magnetic field strength in the solar wind are very similar. These results suggest that the in-flight calibration of the MOMAG is successful and the MOMAG provides reliable data for scientific research. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.04312-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.04312) | **A comprehensive view of the interstellar medium in a quasar host galaxy  at z~6.4**  |
|| Roberto Decarli, et al. -- incl., <mark>Fabian Walter</mark>, <mark>Frank Bertoldi</mark>, <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2023-02-10*|
|*Comments*| *16 pages, 14 figures. Accepted for publication in A&A*|
|**Abstract**| Characterizing the physical conditions (density, temperature, ionization state, metallicity, etc) of the interstellar medium is critical to our understanding of the formation and evolution of galaxies. Here we present a multi-line study of the interstellar medium in the host galaxy of a quasar at z~6.4, i.e., when the universe was 840 Myr old. This galaxy is one of the most active and massive objects emerging from the dark ages, and therefore represents a benchmark for models of the early formation of massive galaxies. We used the Atacama Large Millimeter Array to target an ensemble of tracers of ionized, neutral, and molecular gas, namely the fine-structure lines: [OIII] 88$\mu$m, [NII] 122$\mu$m, [CII] 158$\mu$m, and [CI] 370$\mu$m and the rotational transitions of CO(7-6), CO(15-14), CO(16-15), and CO(19-18); OH 163.1$\mu$m and 163.4$\mu$m; and H$_2$O 3(0,3)-2(1,2), 3(3,1)-4(0,4), 3(3,1)-3(2,2), 4(0,4)-3(1,3), 4(3,2)-4(2,3). All the targeted fine-structure lines are detected, as are half of the targeted molecular transitions. By combining the associated line luminosities, the constraints on the dust temperature from the underlying continuum emission, and predictions from photoionization models of the interstellar medium, we find that the ionized phase accounts for about one third of the total gaseous mass budget, and is responsible for half of the total [CII] emission. It is characterized by high density (n~180 cm$^{-3}$), typical of HII regions. The spectral energy distribution of the photoionizing radiation is comparable to that emitted by B-type stars. Star formation also appears to drive the excitation of the molecular medium. We find marginal evidence for outflow-related shocks in the dense molecular phase, but not in other gas phases. This study showcases the power of multi-line investigations in unveiling the properties of the star-forming medium in galaxies at cosmic dawn. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.04507.md
    + _build/html/tmp_2302.04507/./stacked_biases.png
    + _build/html/tmp_2302.04507/./all_dcls_estimators.png
    + _build/html/tmp_2302.04507/./bias_contours_lcdm.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\orcid}[1]$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\orcid$}[1]$</div>



<div id="title">

#  preparation: XXVIII. Modelling of the weak lensing angular power spectrum

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.04507-b31b1b.svg)](https://arxiv.org/abs/2302.04507)<mark>Appeared on: 2023-02-10</mark> - _20 pages, submitted to A&A_

</div>
<div id="authors">

Euclid Collaboration, et al. -- incl., <mark><mark>M. L. Brown</mark></mark>, <mark><mark>E. Franceschi</mark></mark>, <mark><mark>K. Jahnke</mark></mark>, <mark><mark>S. Ligori</mark></mark>, <mark><mark>P. B. Lilje</mark></mark>

</div>
<div id="abstract">

**Abstract:** This work considers which higher-order effects in modelling the cosmic shear angular power spectra must be taken into account for\emph{Euclid}. We identify which terms are of concern, and quantify their individual and cumulative impact on cosmological parameter inference from\emph{Euclid}. We compute the values of these higher-order effects using analytic expressions, and calculate the impact on cosmological parameter estimation using the Fisher matrix formalism. We review 24 effects and find the following potentially need to be accounted for: the reduced shear approximation, magnification bias, source-lens clustering, source obscuration, local Universe effects, and the flat Universe assumption. Upon computing these explicitly, and calculating their cosmological parameter biases, using a maximum multipole of$\ell=5000$, we find that the magnification bias, source-lens clustering, source obscuration, and local Universe terms individually produce significant ($ >0.25\sigma$) cosmological biases in one or more parameters, and accordingly must be accounted for. In total, over all effects, we find biases in$\Omega_{\rm m}$,$\Omega_{\rm b}$,$h$, and$\sigma_{8}$of$0.73\sigma$,$0.28\sigma$,$0.25\sigma$, and$-0.79\sigma$, respectively, for flat$\Lambda$CDM. For the$w_0w_a$CDM case, we find biases in$\Omega_{\rm m}$,$\Omega_{\rm b}$,$h$,$n_{\rm s}$,$\sigma_{8}$, and$w_a$of$1.49\sigma$,$0.35\sigma$,$-1.36\sigma$,$1.31\sigma$,$-0.84\sigma$, and$-0.35\sigma$, respectively; which are increased relative to the$\Lambda$CDM due to additional degeneracies as a function of redshift and scale.

</div>

<div id="div_fig1">

<img src="tmp_2302.04507/./stacked_biases.png" alt="Fig2" width="100%"/>

**Figure 2. -** Stacked bar chart of cosmological parameter biases resulting from the studied higher-order effects, for the flat $\Lambda$CDM case (left) of Table \ref{tab:biasesLCDMtab}, and the flat $w_0w_a$CDM case (right) of Table \ref{tab:biasesflattab}. The non-flat Universe term is not shown here, due to the different cosmology. Biases are presented here as a fraction of the $1\sigma$ parameter uncertainty. A bias is non-negligble if its absolute value reaches or exceeds $0.25\sigma$. `RS' denotes the reduced shear correction, `SLC' is the source-lens clustering term, `MB' is the magnification bias correction, `SO' is the two-point source obscuration correction, `SO-MB' and `SO-SLC' are the source obscuration-magnification bias and source-lens clustering cross terms respectively, and `LU' is the local Universe correction The segments with the dashed outlines show the total parameter biases from these corrections for each parameter. (*fig:stacked_bars*)

</div>
<div id="div_fig2">

<img src="tmp_2302.04507/./all_dcls_estimators.png" alt="Fig1" width="100%"/>

**Figure 1. -** Absolute magnitudes of the reduced shear, source-lens clustering, magnification bias, local Universe, source obscuration, and non-flat Universe corrections to the shear angular power spectra, relative to those angular power spectra, for \emph{Euclid}. The corrections to the angular power spectra for four redshift bin auto-correlations are shown as representative examples, spanning across the redshift range of \emph{Euclid}. The remaining auto and cross-correlations exhibit the same patterns. The absolute value of the signed sum of the corrections is also shown. These are all compared to the sample variance, calculated according to Eq. (\ref{eq:samplevar}). Notably, while the magnitudes of individual corrections are either higher at lower redshifts or vice-versa, the magnitude of the sum of the corrections is consistently high. Additionally, the cross-terms between source obscuration, and magnification bias and source-lens clustering are multiple orders of magnitude below other terms and sample variance, suggesting they are negligible. The remainder of the terms are typically of similar magnitudes across redshifts, suggesting they must all be accounted for. We note that these magnitudes are for both the $\Lambda$CDM and $w_0w_a$CDM cases, as the choice of fiducial values for the latter matches the former, and that the non-flat Universe correction here has been computed for a cosmology with $\Omega_{K}=0.05$, whilst other corrections are when $\Omega_{K}=0$. The markers for the SO-MB, SO-SLC, SO, Non-flat Universe, and total lines are only used to distinguish those from the other terms, and do not have any other significance. The symbols (points) are only included to allow a reader to distinguish the lines (in particular if printing in gray-scale) and do not indicate the $\ell$-modes where a computation was made; all quantities were evaluated for 200 $\ell$-bands, logarithmically spaced, with an $\ell_{\rm min}$ of 10, and an $\ell_{\rm max}$ of 5000. (*fig:dcls_all*)

</div>
<div id="div_fig3">

<img src="tmp_2302.04507/./bias_contours_lcdm.png" alt="Fig3" width="100%"/>

**Figure 3. -** Projected 1$\sigma$ and 2$\sigma$ 2-parameter uncertainty contours for \emph{Euclid} under a $\Lambda$CDM cosmology, with and without correcting for the source-lens clustering, magnification bias, source obscuration, and local Universe terms. These are predicted using the Fisher matrix formalism, using the cosmology specified in Table \ref{tab:cosmology}, in the case when $\Omega_{K}=0$ and is kept fixed. The true location of the constraints is denoted by the blue, dashed contours, while the biased locations if the corrections are not made are given by the solid, gold contours. Significant biases are predicted for $\Omega_{\rm m}$, $\Omega_{\rm b}$, $h$, and $\sigma_{8}$, and their values can be found in Table \ref{tab:biasesLCDMtab} (*fig:bias_corner_LCDM*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

199  publications in the last 7 days.
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers